# 1. Deep Learning

In [122]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import datetime
from dateutil import rrule

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(1)
seed(1)

### 1. Time Series Visualization with plotly

In [123]:
weekly_sales = pd.read_csv('Console Sales by Week.csv').dropna()
weekly_sales.head()

,Date,Nintendo Switch Sales,PS4 Sales,Xbox One Sales
0,1/6/2018,71528,59920,49805
1,1/13/2018,62374,50554,46845
2,1/20/2018,53734,47961,44867
3,1/27/2018,58457,61473,54336
4,2/3/2018,59834,58728,53374


#### Overall Weekly Sales

In [124]:
product_weekly_sales_sc = []
for product in ['Nintendo Switch Sales', 'PS4 Sales', 'Xbox One Sales']:
    current_product_weekly_sales = weekly_sales[product]
    product_weekly_sales_sc.append(go.Scatter(x = weekly_sales['Date'], y=current_product_weekly_sales * 0.1, name=(product)))

layout = go.Layout(title = 'Store daily sales', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = product_weekly_sales_sc, layout=layout)
iplot(fig)

#### Nintendo Switch Sales

In [125]:
weekly_sales_sc = go.Scatter(x = weekly_sales['Date'], y = weekly_sales['Nintendo Switch Sales'])
layout = go.Layout(title='Weekly Sales', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
fig = go.Figure(data=[weekly_sales_sc], layout=layout)
iplot(fig)

### 2. Train Test Split
Since the sales of Nintendo follow similar pattern across different years, it might not be a good idea to randomly split the dataset. We decided to use the sales from 1/6/2018 to 12/28/2019 as the train set and the sales at the first 4 months in 2020 as the test set.

In [126]:
weekly_sales['Date'] = pd.DatetimeIndex(weekly_sales['Date']) # convert the data type of Date from String to Datetime
weekly_sales['year'] = pd.DatetimeIndex(weekly_sales['Date']).year # extract the year

In [127]:
train = weekly_sales[['Date', 'Nintendo Switch Sales', 'year']]
train = train[train['year'] != 2020].drop(['year'], axis = 1)
train.tail()

,Date,Nintendo Switch Sales
99,2019-11-30,881303
100,2019-12-07,512053
101,2019-12-14,551514
102,2019-12-21,616317
103,2019-12-28,356436


In [128]:
test = weekly_sales[['Date', 'Nintendo Switch Sales', 'year']]
test = test[test['year'] == 2020].drop(['year'], axis = 1)
test.head()

,Date,Nintendo Switch Sales
104,2020-01-04,80316
105,2020-01-11,72995
106,2020-01-18,65624
107,2020-01-25,61031
108,2020-02-01,61915


Identify what's the time gap between the last day of train set from the last day of the test set, this will be out lag (the amount of day that need to be forecast)

In [129]:
lag_size = 16

### 3. Transform The Time Series Dataset to Supervised Problem By Using Shift Method

In [130]:
# Define the function for shifting dataset
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

We will use the current timestep and the last week to forecast 16 weeks ahead

In [131]:
window = 2
lag = lag_size
series = series_to_supervised(train.drop('Date', axis=1), window=window, lag=lag)
series.head(10)

,Nintendo Switch Sales(t-2),Nintendo Switch Sales(t-1),Nintendo Switch Sales(t),Nintendo Switch Sales(t+16)
2,71528.0,62374.0,53734,40765.0
3,62374.0,53734.0,58457,41626.0
4,53734.0,58457.0,59834,42223.0
5,58457.0,59834.0,64734,41793.0
6,59834.0,64734.0,68945,59457.0
7,64734.0,68945.0,65708,64099.0
8,68945.0,65708.0,66374,80980.0
9,65708.0,66374.0,63748,76685.0
10,66374.0,63748.0,59834,63339.0
11,63748.0,59834.0,63831,69671.0


In [132]:
# Label
labels_col = 'Nintendo Switch Sales(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

In [133]:
X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (51, 3)
Validation set shape (35, 3)


,Nintendo Switch Sales(t-2),Nintendo Switch Sales(t-1),Nintendo Switch Sales(t)
53,246756.0,82734.0,68800
73,51024.0,54337.0,58848
77,76542.0,78293.0,107933
57,65730.0,66535.0,78855
6,59834.0,64734.0,68945


### 4. Predictive Model

#### MLP for Time Series Forecasting

In [134]:
epochs = 60
batch = 32
lr = 0.0005
adam = optimizers.Adam(lr)

In [135]:
model_mlp = Sequential()
model_mlp.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))
model_mlp.add(Dense(1))
model_mlp.compile(loss='mae', optimizer=adam)
model_mlp.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               400       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________


In [136]:
mlp_history = model_mlp.fit(X_train.values, Y_train, validation_data=(X_valid.values, Y_valid), epochs=epochs, verbose=2)

Train on 51 samples, validate on 35 samples
Epoch 1/60
 - 0s - loss: 130879.1867 - val_loss: 77566.5420
Epoch 2/60
 - 0s - loss: 130171.2912 - val_loss: 76924.4703
Epoch 3/60
 - 0s - loss: 129428.5461 - val_loss: 76361.7185
Epoch 4/60
 - 0s - loss: 128865.4616 - val_loss: 75819.2833
Epoch 5/60
 - 0s - loss: 128522.4378 - val_loss: 75271.9431
Epoch 6/60
 - 0s - loss: 127815.4395 - val_loss: 74764.1473
Epoch 7/60
 - 0s - loss: 127276.1158 - val_loss: 74243.7681
Epoch 8/60
 - 0s - loss: 126720.2521 - val_loss: 73718.9368
Epoch 9/60
 - 0s - loss: 126118.4021 - val_loss: 73208.8217
Epoch 10/60
 - 0s - loss: 125684.8701 - val_loss: 72679.8306
Epoch 11/60
 - 0s - loss: 125236.6838 - val_loss: 72157.2109
Epoch 12/60
 - 0s - loss: 124791.8658 - val_loss: 71648.3819
Epoch 13/60
 - 0s - loss: 124329.4139 - val_loss: 71144.3658
Epoch 14/60
 - 0s - loss: 124040.7742 - val_loss: 70629.5962
Epoch 15/60
 - 0s - loss: 123567.2595 - val_loss: 70151.6018
Epoch 16/60
 - 0s - loss: 123127.3566 - val_loss: 

#### RNN (LSTM) for Times Series Forecasting

In [137]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_valid_series.shape)

Train set shape (51, 3, 1)
Validation set shape (35, 3, 1)


In [138]:
model_lstm = Sequential()
model_lstm.add(LSTM(100, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer=adam)
model_lstm.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [139]:
lstm_history = model_lstm.fit(X_train_series, Y_train, validation_data=(X_valid_series, Y_valid), epochs=epochs, verbose=2)

Train on 51 samples, validate on 35 samples
Epoch 1/60
 - 0s - loss: 159337.2338 - val_loss: 105265.3828
Epoch 2/60
 - 0s - loss: 158027.7160 - val_loss: 103835.5560
Epoch 3/60
 - 0s - loss: 156719.7178 - val_loss: 102623.2462
Epoch 4/60
 - 0s - loss: 155436.7485 - val_loss: 100946.4462
Epoch 5/60
 - 0s - loss: 154363.1566 - val_loss: 100339.1132
Epoch 6/60
 - 0s - loss: 153200.4709 - val_loss: 99057.1812
Epoch 7/60
 - 0s - loss: 152003.4638 - val_loss: 97883.3426
Epoch 8/60
 - 0s - loss: 150750.9170 - val_loss: 96187.8402
Epoch 9/60
 - 0s - loss: 148969.3822 - val_loss: 94484.7589
Epoch 10/60
 - 0s - loss: 147736.1131 - val_loss: 94062.0042
Epoch 11/60
 - 0s - loss: 146613.3447 - val_loss: 92684.8661
Epoch 12/60
 - 0s - loss: 145465.6841 - val_loss: 91311.7967
Epoch 13/60
 - 0s - loss: 143883.2169 - val_loss: 89931.9391
Epoch 14/60
 - 0s - loss: 142496.1621 - val_loss: 88539.8511
Epoch 15/60
 - 0s - loss: 141276.8487 - val_loss: 87217.2717
Epoch 16/60
 - 0s - loss: 140020.4773 - val_l

### 5. Sales Forecasting

In [140]:
X_pred = pd.DataFrame({'Nintendo Switch Sales(t-2)}': [72995],
                      'Nintendo Switch Sales(t-1)}': [65624],
                      'Nintendo Switch Sales(t)}': [61031]})

X_predict = X_pred.values.reshape((X_pred.shape[0], X_pred.shape[1], 1))

In [141]:
# sales predicted by LSTM = 64075.83 (since the LSTM model has random process, it may produce different values in different trials)
model_lstm.predict(X_predict)

array([[55560.254]], dtype=float32)

In [142]:
# sales predicted by MLP = 57486.945
model_mlp.predict(X_pred)

array([[57400.406]], dtype=float32)

### 6. Data Visulization of Model Performance

In [143]:
MLP_prediction = pd.DataFrame(model_mlp.predict(series))


In [144]:
train = train.drop([0,1])
train = train.reset_index(drop = True)


#### MLP Performance

In [145]:
MLP_df = pd.concat([train, MLP_prediction], axis=1)
MLP_df.rename(columns={'Nintendo Switch Sales':'actual sales',0:'predicted sales'}, inplace=True)

In [146]:
MLP_df = MLP_df[MLP_df.index <= 114]

In [1]:
MLP_sc = []
for values in ['actual sales', 'predicted sales']:
    current_sales = MLP_df[values]
    MLP_sc.append(go.Scatter(x = MLP_df['Date'], y=current_sales, name=(values)))

layout = go.Layout(title = 'MLP Prediction', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = MLP_sc, layout=layout)
iplot(fig)

NameError: name 'MLP_df' is not defined

#### LSTM Performance

In [148]:
series = series.values.reshape((series.shape[0], series.shape[1], 1))

In [149]:
LSTM_prediction = pd.DataFrame(model_lstm.predict(series))
LSTM_df = pd.concat([train, LSTM_prediction], axis=1)
LSTM_df.rename(columns={'Nintendo Switch Sales':'actual sales',0:'predicted sales'}, inplace=True)

In [150]:
LSTM_sc = []
for values in ['actual sales', 'predicted sales']:
    current_sales = LSTM_df[values]
    LSTM_sc.append(go.Scatter(x = LSTM_df['Date'], y=current_sales, name=(values)))

layout = go.Layout(title = 'LSTM Prediction', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = LSTM_sc, layout=layout)
iplot(fig)

### 7. Deep Learning Conclusion

MLP: Prediction = 57486.945; MAE = 64821
<br>
LSTM: Prediction = 64075.83; MAE = 61376

# 2. Machine Learning

In [151]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import plot_importance
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn import preprocessing

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(1)
seed(1)

### 1. Load And Modify Data

In [152]:
weekly_sales = pd.read_csv('Console Sales by Week.csv').dropna()
weekly_sales = pd.read_csv('Console Sales by Week.csv').dropna()
weekly_sales['Date'] = pd.DatetimeIndex(weekly_sales['Date']) # convert the data type of Date from String to Datetime
weekly_sales['year'] = pd.DatetimeIndex(weekly_sales['Date']).year 
weekly_sales['month'] = pd.DatetimeIndex(weekly_sales['Date']).month 
weekly_sales['week'] = pd.DatetimeIndex(weekly_sales['Date']).week
weekly_sales['day'] = pd.DatetimeIndex(weekly_sales['Date']).day
weekly_sales = weekly_sales.drop(['Date'], axis=1)
weekly_sales.head()

,Nintendo Switch Sales,PS4 Sales,Xbox One Sales,year,month,week,day
0,71528,59920,49805,2018,1,1,6
1,62374,50554,46845,2018,1,2,13
2,53734,47961,44867,2018,1,3,20
3,58457,61473,54336,2018,1,4,27
4,59834,58728,53374,2018,2,5,3


### 2. Data Preprocessing and Feature Engineering

In [153]:
X = weekly_sales.drop(['Nintendo Switch Sales', 'PS4 Sales', 'Xbox One Sales'], axis=1)
y = weekly_sales['Nintendo Switch Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

### 3. Predictive Model

#### Random Forest and XGBoost

In [154]:
regressors = [
    RandomForestRegressor(max_depth=10, random_state=1),
    XGBRegressor(max_depth=10, learning_rate=0.05, random_state=1)]

In [155]:
MAE = []
for regressor in regressors:
    fit = regressor.fit(X_train, y_train)
    pred = fit.predict(X_test)
    mae = mean_absolute_error( y_test,pred)
    MAE.append(mae)
MAE 

[16:51:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[47193.78952380952, 52083.1259765625]

#### LightGBM

In [156]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 10,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [157]:
gbm = lgb.train(params,
                train_set=lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=300)

[1]	valid_0's l1: 85753.2	valid_0's l2: 3.25048e+10
Training until validation scores don't improve for 300 rounds
[2]	valid_0's l1: 85670.2	valid_0's l2: 3.21243e+10
[3]	valid_0's l1: 85591.4	valid_0's l2: 3.178e+10
[4]	valid_0's l1: 85632.8	valid_0's l2: 3.14687e+10
[5]	valid_0's l1: 85737.6	valid_0's l2: 3.1187e+10
[6]	valid_0's l1: 85575.3	valid_0's l2: 3.09221e+10
[7]	valid_0's l1: 85417.2	valid_0's l2: 3.0682e+10
[8]	valid_0's l1: 85273.8	valid_0's l2: 3.04627e+10
[9]	valid_0's l1: 85134	valid_0's l2: 3.02636e+10
[10]	valid_0's l1: 85007.4	valid_0's l2: 3.00818e+10
[11]	valid_0's l1: 85258	valid_0's l2: 2.98727e+10
[12]	valid_0's l1: 85511.3	valid_0's l2: 2.96869e+10
[13]	valid_0's l1: 85740.3	valid_0's l2: 2.95201e+10
[14]	valid_0's l1: 85972.4	valid_0's l2: 2.93725e+10
[15]	valid_0's l1: 86182	valid_0's l2: 2.92397e+10
[16]	valid_0's l1: 85721.1	valid_0's l2: 2.92095e+10
[17]	valid_0's l1: 85289.8	valid_0's l2: 2.91831e+10
[18]	valid_0's l1: 84880.1	valid_0's l2: 2.916e+10
[19]	

In [158]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

MAE.append(mean_absolute_error(y_test, y_pred))
MAE

[47193.78952380952, 52083.1259765625, 81953.85695471083]

### 4. Sales Forecasting

In [159]:
x = datetime.datetime(2020, 5, 9)
X_pred = pd.DataFrame({'year': [x.year], 'month':[x.month], 
                       'week':[x.isocalendar()[1]], 'day':[x.day]})

In [160]:
# sales predicted by LightGBM = 65338.12124415
gbm.predict(X_pred, num_iteration=gbm.best_iteration)

array([65338.12124415])

In [161]:
# sales predicted by XGBoost = 51551.793
fit.predict(X_pred)

array([51551.793], dtype=float32)

In [162]:
# sales predicted by RandomForest = 51409.96666667
rf = RandomForestRegressor(max_depth=10, random_state=1)
rf.fit(X_train, y_train)

rf.predict(X_pred)

array([51409.96666667])

### 5. Data Visualization of Model Performance

#### LightGBM Performance

In [163]:
weekly_sales = pd.read_csv('Console Sales by Week.csv').dropna()
gbm_prediction = pd.DataFrame(gbm.predict(X, num_iteration=gbm.best_iteration))
gbm_df = pd.concat([weekly_sales, gbm_prediction], axis=1)
gbm_df.rename(columns={'Nintendo Switch Sales':'actual sales',0:'predicted sales'}, inplace=True)

In [164]:
gbm_sc = []
for values in ['actual sales', 'predicted sales']:
    current_sales = gbm_df[values]
    gbm_sc.append(go.Scatter(x = gbm_df['Date'], y=current_sales, name=(values)))

layout = go.Layout(title = 'LightGBM Prediction', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = gbm_sc, layout=layout)
iplot(fig)

#### RandomForest Performance

In [165]:
rf_prediction = pd.DataFrame(rf.predict(X))
rf_df = pd.concat([weekly_sales, rf_prediction], axis=1)
rf_df.rename(columns={'Nintendo Switch Sales':'actual sales',0:'predicted sales'}, inplace=True)

In [166]:
rf_sc = []
for values in ['actual sales', 'predicted sales']:
    current_sales = rf_df[values]
    rf_sc.append(go.Scatter(x = rf_df['Date'], y=current_sales, name=(values)))

layout = go.Layout(title = 'Random Forest Prediction', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = rf_sc, layout=layout)
iplot(fig)

#### XGB Performance

In [167]:
XGB_prediction = pd.DataFrame(fit.predict(X))
XGB_df = pd.concat([weekly_sales, XGB_prediction], axis=1)
XGB_df.rename(columns={'Nintendo Switch Sales':'actual sales',0:'predicted sales'}, inplace=True)

In [168]:
XGB_sc = []
for values in ['actual sales', 'predicted sales']:
    current_sales = XGB_df[values]
    XGB_sc.append(go.Scatter(x = XGB_df['Date'], y=current_sales, name=(values)))

layout = go.Layout(title = 'XGB Prediction', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = XGB_sc, layout=layout)
iplot(fig)

### 6. Machine Learning Conclusion

RandomForest: Prediction = 51409.96666667; MAE = 47193
<br>
XGBoost: Prediction = 51551.793; MAE = 52083

# 3. Final Prediction

### 1. Collect Predicted sales of RandomForest, XGBoost, MLP, and LSTM

In [171]:
RF_Pred = 51410
XGB_Pred = 51552
MLP_Pred = 57487
LSTM_Pred = 64076

### 2. Calculate Weights for RandomForest, XGBoost, MLP, and LSTM

In [172]:
MAE = [47193, 52083, 64821, 61376]
Weights = [float(i)/sum(MAE) for i in MAE]
RF_Weight = Weights[0]
XGB_Weight = Weights[1]
MLP_Weight = Weights[2]
LSTM_Weight = Weights[3]

### 3. Calculate Final Prediction

In [174]:
Predicted_Sales = RF_Pred * RF_Weight + XGB_Pred * XGB_Weight + MLP_Pred * MLP_Weight + LSTM_Pred * LSTM_Weight
Predicted_Sales

56637.68322149437